In [1]:
import pandas as pd
df = pd.read_csv('data/BTC_final.csv')

In [2]:
df.head()

,Unnamed: 0,Open,High,Low,Close,Volume,CionScore,CoinDate
0,0,387.427002,391.378998,380.779999,383.614990,26229400.0,-0.224409,2014-10-01
1,1,383.988007,385.497009,372.946014,375.071991,21777700.0,0.162439,2014-10-02
2,2,375.181000,377.695007,357.859009,359.511993,30901200.0,-0.136249,2014-10-03
3,3,359.891998,364.487000,325.885986,328.865997,47236500.0,-0.136249,2014-10-04
4,4,328.915985,341.800995,289.295990,320.510010,83308096.0,-0.136249,2014-10-05


In [ ]:
df['CionScore']